In [1]:
import csv
import json
import pandas as pd
import seaborn as sns
import tkinter as tk
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.create_widgets()
        self.loaded = False


    def create_widgets(self):
        self.text = tk.StringVar()
        self.text.set("Welcome")
        self.label = tk.Label(textvariable=self.text)        
        self.label.pack(side="top")
        
        self.load_initial = tk.Button(self)
        self.load_initial["text"] = "Load the initial csv data set"
        self.load_initial["command"] = self.load_csv_data
        self.load_initial.pack(side="top")
        
        self.clean = tk.Button(self)
        self.clean["text"] = "Preprocess the initial data set"
        self.clean["command"] = self.preprocess
        self.clean.pack(side="top")
        
        self.save = tk.Button(self)
        self.save["text"] = "Save to JSON"
        self.save["command"] = self.save_dataframe_to_json
        self.save.pack(side="top")
        
        self.load = tk.Button(self)
        self.load["text"] = "Load the JSON data set"
        self.load["command"] = self.load_json_data
        self.load.pack(side="top")
        
        self.visual = tk.Button(self)
        self.visual["text"] = "Produce graph"
        self.visual["command"] = self.visualize
        self.visual.pack(side="top")
        
        self.determine = tk.Button(self)
        self.determine["text"] = "Determine relationship"
        self.determine["command"] = self.correlate
        self.determine.pack(side="top")

        self.quit = tk.Button(self, text="QUIT", fg="red",
                              command=self.master.destroy)
        self.quit.pack(side="bottom")


    def load_csv_data(self):
        try:
            self.Inspections = pd.read_csv(r'Inspections.csv')
            self.Inventroy = pd.read_csv(r'Inventroy.csv', nrows=100)
            self.violations = pd.read_csv(r'violations.csv')
            self.text.set("Loaded CSV")
            self.loaded = True
        except:
            self.text.set("Error loading data. Please check that files are present")
        
        
    def save_dataframe_to_json(self):
        if self.loaded:
            self.Inspections.to_json(r'Inspections.json')
            self.Inventroy.to_json(r'Inventroy.json')
            self.violations.to_json(r'violations.json')
            self.text.set("Saved to JSON")
        else:
            self.text.set("Error. Please load data from files")
        
        
    def load_json_data(self):
        try:
            self.Inspections = pd.read_json(r'Inspections.json')
            self.Inventroy = pd.read_json(r'Inventroy.json')
            self.violations = pd.read_json(r'violations.json')
            self.text.set("Loaded JSON")
            self.loaded = True
        except:
            self.text.set("Error loading data. Please check that files are present")
    
    
    def preprocess(self):
        if not self.loaded:            
            self.text.set("Error. Please load data from files")
            return
        
        self.Inspections = self.Inspections[self.Inspections['PROGRAM STATUS'] != 'INACTIVE']
        seating = []
        for description in self.Inspections['PE DESCRIPTION']:
            start = description.find('(')+1
            end = description.find(')')
            seating.append(description[start:end])

        self.Inspections['SEATING'] = seating
        self.Inspections['PE DESCRIPTION'] = self.Inspections['PE DESCRIPTION'].str.replace(r"\(.*\)","")
        
        seating_mean = self.Inspections.groupby('PE DESCRIPTION').mean()
        seating_median = self.Inspections.groupby('PE DESCRIPTION').median()
        seating_mode = self.Inspections.groupby('PE DESCRIPTION').agg(pd.Series.mode)
        
        zip_mean = self.Inspections.groupby('Zip Codes').mean()
        zip_median = self.Inspections.groupby('Zip Codes').median()
        zip_mode = self.Inspections.groupby('Zip Codes').agg(pd.Series.mode)
        
        self.text.set("Data preprocessed")
        
        print(seating_mean)
        print(seating_median)
        print(seating_mode)
        
        print(zip_mean)
        print(zip_median)
        print(zip_mode)
        
        
    def visualize(self):        
        if not self.loaded:            
            self.text.set("Error. Please load data from files")
            return
        
        violations_count = self.violations.groupby('VIOLATION CODE').count()
        violations_count.reset_index(level=0, inplace=True)
        ax = sns.barplot(x='VIOLATION CODE', y='POINTS',
                    data = violations_count)
        #violations_count.index,y=violations_count["POINTS"])
        #plt.rcParams['figure.figsize'] = (100.0,100.0)
        plt.show()
        self.text.set("Bar Graph generated")
        
        
    def correlate(self):        
        if not self.loaded:            
            self.text.set("Error. Please load data from files")
            return
        
        violations_count = self.violations[['SERIAL NUMBER', 'POINTS']].groupby('SERIAL NUMBER').count()
        violations_count.reset_index(level=0, inplace=True)
        df = self.Inspections[['SERIAL NUMBER','Zip Codes','FACILITY ID']].set_index('SERIAL NUMBER').join(
            violations_count.set_index('SERIAL NUMBER'))
        df = df.dropna()
        df.reset_index(level=0, inplace=True)
        df = df.groupby(['Zip Codes','FACILITY ID']).sum().reset_index()
        df = df[['Zip Codes','POINTS']]
        
        #Linear Regression
        X = df.iloc[:, 0].values.reshape(-1, 1)
        Y = df.iloc[:, 1].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        Y_pred = linear_regressor.predict(X)
        
        #Plot the linear regression
        plt.scatter(X, Y)
        plt.plot(X, Y_pred, color='red')
        plt.show()
        
        self.text.set("Scatter Plot generated")
        
root = tk.Tk()
app = Application(master=root)
app.mainloop()

In [58]:
Inspections = pd.read_csv(r'Inspections.csv', nrows=100)
Inventroy = pd.read_csv(r'Inventroy.csv', nrows=100)
violations = pd.read_csv(r'violations.csv', nrows=100)
Inspections.head()
# Inventroy.head()

# violations_count = violations[['SERIAL NUMBER', 'POINTS']].groupby('SERIAL NUMBER').count()
# violations_count.reset_index(level=0, inplace=True)
# violations_count.head()

,ACTIVITY DATE,OWNER ID,OWNER NAME,FACILITY ID,FACILITY NAME,RECORD ID,PROGRAM NAME,PROGRAM STATUS,PROGRAM ELEMENT (PE),PE DESCRIPTION,...,SERVICE DESCRIPTION,SCORE,GRADE,SERIAL NUMBER,EMPLOYEE ID,Location,2011 Supervisorial District Boundaries (Official),Census Tracts 2010,Board Approved Statistical Areas,Zip Codes
0,08/23/2018,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,ROUTINE INSPECTION,97,A,DA2FXQNN6,EE0000126,POINT (-118.36927 33.826754),4.0,820.0,102.0,25719.0
1,12/06/2017,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,ROUTINE INSPECTION,95,A,DACP43IQW,EE0000126,POINT (-118.36927 33.826754),4.0,820.0,102.0,25719.0
2,06/23/2017,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,ROUTINE INSPECTION,96,A,DAEMVMRBY,EE0000126,POINT (-118.36927 33.826754),4.0,820.0,102.0,25719.0
3,03/19/2019,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,ROUTINE INSPECTION,96,A,DANER68S4,EE0000126,POINT (-118.36927 33.826754),4.0,820.0,102.0,25719.0
4,03/01/2018,OW0000002,#1 CAFE INC,FA0056432,#1 CAFE,PR0045100,#1 CAFE,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,ROUTINE INSPECTION,90,A,DACZXQ74W,EE0000015,POINT (-118.412323 34.058815),3.0,660.0,265.0,24032.0
